## Expressive Summary Fucntion

In [220]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import heapq
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
from textblob import TextBlob

In [221]:
nlp = spacy.load("en_core_web_sm")
punctuation = punctuation+"\n"+"“" + '”' +"–"
stopWords = set(stopwords.words("english"))

## Spacy library text summary

In [222]:
def summary_spacy(text,percentage=0.4):
    doc = nlp(text)
    word_freq = Counter(
     word.text.lower() for word in doc 
        if word.text.lower() not in STOP_WORDS 
        if word.text.lower() not in punctuation 
    )
    max_freq = max(word_freq.values())
    word_freq = {k : v/max_freq for k,v in word_freq.items()}
    sentence = [sent for sent in doc.sents]
    sent_score = {}
    for sent in sentence:
        for word in sent:
            if word.text.lower() in word_freq:
                if sent not in sent_score:
                    sent_score[sent] = word_freq[word.text.lower()]
                else:
                    sent_score[sent] += word_freq[word.text.lower()]
    summary = "".join(list(map(lambda x:x.text.strip().replace("\n","") ,
              nlargest(int(len(sent_score)*percentage),# getting 40% summary
                 sent_score,    key=sent_score.get))))
    return summary

## NLTK text summary

In [223]:
def summary_nltk(text,top_largest=3):
    word_frequencies = {}

    for word in nltk.word_tokenize(text):  
        if word not in stopWords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())
    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_list = nltk.sent_tokenize(text)
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]



    summary_sentences = heapq.nlargest(top_largest, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences).strip().replace("\n","")
    return summary

## Latent Sematice analysis text summary

In [224]:
def load_data(document: str):
    doc = TextBlob(document)
    corpus_sentence = doc.sentences
    documents_list = []
    for line in corpus_sentence:
        text = line.strip()
        if len(text) < 1:
            continue
        documents_list.append(str(text))
    print("Total Number of Documents:",len(documents_list))
    return documents_list


def preprocess_data(document_list :List[str])-> List[str]:
    # intialize regex tokenizer
    tokenizer =  RegexpTokenizer(r"\w+")
    # stop words
    en_stop = set(stopwords.words("english"))
    # intialsing portstemmer
    portstemer = PorterStemmer()

    texts = []
    # loop through document list
    for text in document_list:
        # tokenizing data with only words
        tokens  = tokenizer.tokenize(text.lower())
        
        # remove stop words from tokens
        tokens = [word for word in tokens if word not in en_stop]
        
        # stem tokens
        tokens = [portstemer.stem(word) for word in tokens]
        # add tokens to list
        texts.append(tokens)
    return texts

def prepare_corpus(doc_clean):
    """
    Input: clean document
    Output : dictionary and Document Term Matrix
    """
    
    # calculation term dictionary of corpus
    dictionary = corpora.Dictionary(doc_clean)
    # document term matrix
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    return dictionary, doc_term_matrix


def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix = prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    #print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel


def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


def takenext(elem):
        """
        sort
        """
        return elem[1]

def selectTopSent(summSize, numTopics, vecsSort):
    topSentences = []
    sent_no = []
    sentIndexes = set()
    sCount = 0
    for i in range(summSize):
        for j in range(numTopics):
            vecs = vecsSort[j]
            si = vecs[i][0]
            if si not in sentIndexes:
                sent_no.append(si)
                sCount +=1
                topSentences.append(vecs[i])
                sentIndexes.add(si)
                if sCount == summSize:
                    sent_no
        return sent_no    

    
def summary_latent_semantic_analysis(text,number_of_topics=2,words=50):
    # sentence document
    document_list = load_data(text)
    # cleanup data
    clean_text = preprocess_data(document_list)
    # corpus_dict and document term matrix
    dict1, doc_term_matrix = prepare_corpus(clean_text)
    # using genensime lsa model
    model = create_gensim_lsa_model(clean_text, number_of_topics, words)
    corpus_lsi = model[doc_term_matrix]
    #sort each vector by score
    vecsSort = list(map(lambda i: list(), range(number_of_topics)))
    # adding vecssort with value
    for i,docv in enumerate(corpus_lsi):
        for sc in docv:
            isent = (i, abs(sc[1]))
            vecsSort[sc[0]].append(isent)
    # sorting vector according to value
    vecsSort = list(map(lambda x: sorted(x,key=takenext,reverse=True), vecsSort))
    topSentences = selectTopSent(8, 2, vecsSort)
    topSentences.sort()
    summary = []
    doc = []
    for idx ,sentence in enumerate(document_list):
        if idx in topSentences:
            summary.append(sentence)   
    summary = " ".join(summary)
    return summary

In [225]:
new_text = """Marathi (English: /məˈrɑːti/;[5] मराठी, Marāṭhī, Marathi: [məˈɾaːʈʰiː] (About this soundlisten)) is an Indo-Aryan language spoken predominantly by around 83 million Marathi people of Maharashtra, India. It is the official language and co-official language in the Maharashtra and Goa states of Western India, respectively and is one of the 22 scheduled languages of India. With 83 million speakers as 2011, Marathi ranks 10th in the list of languages with most native speakers in the world. Marathi has the third largest number of native speakers in India, after Hindi and Bengali.[6] The language has some of the oldest literature of all modern Indian languages, dating from around 600 AD.[7] The major dialects of Marathi are Standard Marathi and the Varhadi dialect.[8] Koli, Agri and Malvani Konkani have been heavily influenced by Marathi varieties.Marathi distinguishes inclusive and exclusive forms of 'we' and possesses a three-way gender system that features the neuter in addition to the masculine and the feminine."""

In [226]:
summary_spacy(new_text)

'Marathi (English: /məˈrɑːti/;[5] मराठी, Marāṭhī, Marathi: [məˈɾaːʈʰiː] (About this soundlisten)) is an Indo-Aryan language spoken predominantly by around 83 million Marathi people of Maharashtra, India.The major dialects of Marathi are Standard Marathi and the Varhadi dialect.[8] Koli, Agri and Malvani Konkani have been heavily influenced by Marathi varieties.'

In [227]:
summary_nltk(new_text)

'Marathi (English: /məˈrɑːti/;[5] मराठी, Marāṭhī, Marathi: [məˈɾaːʈʰiː] (About this soundlisten)) is an Indo-Aryan language spoken predominantly by around 83 million Marathi people of Maharashtra, India. [6] The language has some of the oldest literature of all modern Indian languages, dating from around 600 AD. With 83 million speakers as 2011, Marathi ranks 10th in the list of languages with most native speakers in the world.'

In [228]:
summary_latent_semantic_analysis(new_text)

Total Number of Documents: 7


'Marathi (English: /məˈrɑːti/;[5] मराठी, Marāṭhī, Marathi: [məˈɾaːʈʰiː] (About this soundlisten)) is an Indo-Aryan language spoken predominantly by around 83 million Marathi people of Maharashtra, India. It is the official language and co-official language in the Maharashtra and Goa states of Western India, respectively and is one of the 22 scheduled languages of India.'